# 1. Introduction: What is an RBF Network?

In [1]:
# Import required libraries

# Avoid KMeans memory leak issue on Windows with Intel MKL
import os
os.environ['OMP_NUM_THREADS'] = '1'
import warnings
warnings.filterwarnings('ignore', message='KMeans is known to have a memory leak')
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, confusion_matrix

np.random.seed(42)
print("Libraries loaded successfully!")

Libraries loaded successfully!


# 2. The Gaussian (RBF) Activation Function